In [6]:
import pandas as pd
import numpy as np

# Combine all the csv's into one dataframe
def compile_csv(startYear, endYear):
    df_list = []
    for i in range(startYear, endYear + 1):
        off_filename = "Data/"+ str(i) + 'Offense.csv'
        def_filename = "Data/" + str(i) + 'Defense.csv'
        off_df = pd.read_csv(off_filename)
        off_df["Unit"] = ["Offense" if pos != "LS" else "Special" for pos in off_df["Pos"]]
        def_df = pd.read_csv(def_filename)
        def_df["Unit"] = ["Defense" if (pos != "K" and pos != "P") else "Special" for pos in def_df["Pos"]]
        df_list.append(off_df)
        df_list.append(def_df)
    data = pd.concat(df_list)
    return data

df = compile_csv(2000, 2017)
# Format player name
df["Player"] = [x.split("\\")[0] for x in df["Player"]]

# Parse out Drafted (tm/rnd/yr) column
df["Drafted (tm/rnd/yr)"] = df["Drafted (tm/rnd/yr)"].where(pd.notnull(df["Drafted (tm/rnd/yr)"]), None)
df["DraftTeam"] = [x.split(" / ")[0] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = [x.split(" / ")[1] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = df["DraftRd"].str.replace('[a-zA-Z]+', '')
df["DraftPick"] = [x.split(" / ")[2] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftPick"] = df["DraftPick"].str.replace('[a-zA-Z_]+', '')
df = df.drop(["Drafted (tm/rnd/yr)"], axis=1)

# Convert height to inches
def convert_height(x):
    feet = x.split("-")[0]
    inches = x.split("-")[1]
    height = (int(feet) * 12) + int(inches)
    return height
df['Height'] = df['Height'].apply(convert_height)

df

df.to_csv('Data/All_Data.csv', index=False)

In [4]:
# Strip out unneeded columns and handle undrafted players
df2 = df[['Year','Pos', 'Height', 'Wt', '40YD', 'Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle','Unit', 'DraftRd']]
df2['DraftRd'] = df2['DraftRd'].fillna(8)

# impute missing values for events which players skipped with a -1
df2 = df2.fillna('-1')
df2 = df2.apply(pd.to_numeric, errors='ignore')

# calculate quartile in each event for each player within their combine year/position 
rank_40 = []
rank_vert = []
rank_bench = []
rank_broad = []
rank_3cone = []
rank_shuttle = []
for index, row in df2.iterrows():
    year = row['Year']
    position = row['Pos']
    #Calculate 40YD quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.75)
    quartile = 4
    if row['40YD'] == -1:
        quartile = 5
    elif row['40YD'] <= quartile_1:
        quartile = 1
    elif row['40YD'] <= median:
        quartile = 2
    elif row['40YD'] <= quartile_3:
        quartile = 3
    rank_40.append(quartile)
    
    #Calculate vert quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.75)
    quartile = 4
    if row['Vertical'] == -1:
        quartile = 5
    elif row['Vertical'] >= quartile_3:
        quartile = 1
    elif row['Vertical'] >= median:
        quartile = 2
    elif row['Vertical'] >= quartile_1:
        quartile = 3
    rank_vert.append(quartile)
    
    #Calculate BenchReps quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.75)
    quartile = 4
    if row['BenchReps'] == -1:
        quartile = 5
    elif row['BenchReps'] >= quartile_3:
        quartile = 1
    elif row['BenchReps'] >= median:
        quartile = 2
    elif row['BenchReps'] >= quartile_1:
        quartile = 3
    rank_bench.append(quartile)
    
    #Calculate Broad Jump quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.75)
    quartile = 4
    if row['Broad Jump'] == -1:
        quartile = 5
    elif row['Broad Jump'] >= quartile_3:
        quartile = 1
    elif row['Broad Jump'] >= median:
        quartile = 2
    elif row['Broad Jump'] >= quartile_1:
        quartile = 3
    rank_broad.append(quartile)
    
    #Calculate 3Cone quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.75)
    quartile = 4
    if row['3Cone'] == -1:
        quartile = 5
    elif row['3Cone'] <= quartile_1:
        quartile = 1
    elif row['3Cone'] <= median:
        quartile = 2
    elif row['3Cone'] <= quartile_3:
        quartile = 3
    rank_3cone.append(quartile)
    
    #Calculate Shuttle quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.75)
    quartile = 4
    if row['Shuttle'] == -1:
        quartile = 5
    elif row['Shuttle'] <= quartile_1:
        quartile = 1
    elif row['Shuttle'] <= median:
        quartile = 2
    elif row['Shuttle'] <= quartile_3:
        quartile = 3
    rank_shuttle.append(quartile)
df2['40_quartile_yr_pos'] = rank_40 
df2['vert_quartile_yr_pos'] = rank_vert
df2['bench_quartile_yr_pos'] = rank_bench
df2['broad_quartile_yr_pos'] = rank_broad
df2['3cone_quartile_yr_pos'] = rank_3cone
df2['shuttle_quartile_yr_pos'] = rank_shuttle
df2

/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_2576/2889184105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DraftRd'] = df2['DraftRd'].fillna(8)
/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_2576/2889184105.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2 = df2.apply(pd.to_numeric, errors='ignore')


,Year,Pos,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Unit,DraftRd,40_quartile_yr_pos,vert_quartile_yr_pos,bench_quartile_yr_pos,broad_quartile_yr_pos,3cone_quartile_yr_pos,shuttle_quartile_yr_pos
0,2000,WR,75,191,4.53,33.0,-1.0,130.0,7.09,4.18,Offense,8,2,3,5,1,3,3
1,2000,QB,75,229,4.91,34.0,-1.0,108.0,7.71,4.59,Offense,6th,2,1,5,2,4,4
2,2000,WR,71,180,4.59,36.0,-1.0,123.0,7.22,4.16,Offense,6th,3,2,5,1,4,2
3,2000,OT,76,332,5.26,29.0,28.0,97.0,7.78,4.72,Offense,2nd,2,1,1,3,2,1
4,2000,RB,71,193,4.50,-1.0,-1.0,-1.0,-1.00,-1.00,Offense,5th,1,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2017,DT,73,319,5.04,28.5,25.0,108.0,7.73,4.65,Defense,8,2,1,2,2,4,3
145,2017,DE,79,289,4.83,30.0,23.0,128.0,7.46,4.62,Defense,8,3,4,3,1,4,4
146,2017,DE,75,264,4.79,32.5,14.0,114.0,7.25,4.43,Defense,8,2,2,4,3,3,3
147,2017,DE,76,248,4.61,35.0,30.0,123.0,6.94,4.40,Defense,8,1,1,1,2,1,2
